# Classifying Digits: A Beginner's Tutorial
* In this notebook we use a CNN to train and classify images to 10 different classes (handwritten digits 0-9).

## Table of Contents
* [Prepare Data](#Prepare-Data)
* [Normalize](#Normalize)
* [Reshaping](#Reshaping)
* [One Hot Encoding](#One-Hot-Encoding)
* [Training & Testing](#Training-Testing)
* [Visualizing Images](#Visualizing-Images)
* [CNN](#CNN)
* [Predictions](#Predictions)


In [ ]:
#import required libraries
import pandas as pd
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

## Prepare Data

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
#let's check out our data
train.head()

Above we see that the dataframe has 785 columns with 1 column referring to the label. Our data will take the shape 28X28 = 784 when visualizing images. The label column refers to what class each digit takes. (i.e. digit 5 will be in class 5 and vice-versa). To proceed we will visualize the number of values per digit and also create our y column ('label').

In [ ]:
#init our training and testing
y_train = train['label']
X_train = train.drop(labels = ['label'], axis = 1)

In [ ]:
#How many classes do we have.
print(train['label'].unique())
print('The number of classes is: ' + str(len(train['label'].unique())))

In [ ]:
y_train.value_counts()

For the most part, our labels look to be pretty evenly distributed. The next step is to look at whether there are any missing values in the data.

In [ ]:
print('Number of missing values is: ' + str(X_train.isnull().any().sum()))

## Normalize
* Our machine learning algorithm will perform much better if we normalize the data. We can look at what range the values take, however in this dataset they are images and we have pixel values ranging from 0-255 inclusively. Therefore if we divide each value by 255.0 we will normalize all of our data to be on a 0-1 scale which will help out our classification model a lot.

In [ ]:
print('Range is: ' + str(X_train.values.min()) + '-' + str(X_train.values.max()))

In [ ]:
X_train = X_train / 255.0
test = test / 255.0
X_train = X_train.astype('float32')
test = test.astype('float32')

## Reshaping 
* We must reshape our data because if we investigate the shape of our training dataset we see that there are 784 columns. In order for our algorithm to classify our data we must reshape it into 28X28 values.

In [ ]:
X_train.shape

In [ ]:
X_train = X_train.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

## One Hot Encoding
* We will use keras to_categorical to one hot encode our labels. Therefore it will create an array of length 10 with a 1 representing True. Each position (index) in the array will correspond to the class. So if we have a 1 in index 2 then that will represent a classification of class 2 (assuming index 0 represents class 0). 

In [ ]:
y_train = to_categorical(y_train, num_classes = 10)

## Training-Testing

In [ ]:
#splitting into training and validation sets to measure model performance.
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.1, random_state = 42)

## Visualizing Images
* It will be helpful to visualize our images so we can understand what data will be fed into our algorithm

In [ ]:
X_train.shape

In [ ]:
#look at data shape above. We choose 1st row, then all 28 X 28 pixel values and then 1 to allow matplotlib to identify the image.
img = plt.imshow(X_train[0][:,:,0])

## CNN

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax")) #10 classes 

In [ ]:
model.compile (optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
#can use GPU to speed up training times here.
history = model.fit(X_train, y_train,
                   batch_size = 100,
                    epochs = 20,
                    validation_data = (X_valid, y_valid),
                    verbose = 1,
                   steps_per_epoch = X_train.shape[0] // 100)

We can easily plot our training and validation accuracy & loss to show whether or not our model overfit.

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (10,5))

#accuracy plot
ax[0].plot(history.history['accuracy'], color = 'b', label = 'Training Accuracy', marker = 'o')
ax[0].plot(history.history['val_accuracy'], color = 'r', label = 'Validation Accuracy', marker = 'o')
ax[0].set_title('Accuracy')
ax[0].legend(loc = 'best')

#loss plot
ax[1].plot(history.history['loss'], color = 'b', label = 'Training Loss', marker = 'o')
ax[1].plot(history.history['val_loss'], color = 'r', label = 'Validation Loss', marker = 'o')
ax[1].set_title('Loss')
ax[1].legend(loc = 'best')

## Predictions

In [ ]:
predicted_classes = model.predict_classes(test)
predicted_classes = predicted_classes.reshape(-1,1)

In [ ]:
submission_df = pd.DataFrame(predicted_classes)
submission_df.index.rename('ImageId', inplace = True)
submission_df.index+=1
cols = ['Label']
submission_df.columns = [i for i in cols]

In [ ]:
#write file to csv output
submission_df.to_csv('submission.csv')

In [ ]:
submission_df